In [1]:
%load_ext autoreload
%autoreload 2
import multiprocessing as mp

from output_check2 import output_check
from synthpop.recipes.starter6 import Starter
starter_ver='6'

from synthpop.synthesizer import synthesize_all, enable_logging 
import pandas as pd
import os,time,datetime
import pickle

import gc
import numpy as np
import random

np.random.seed(1)


In [2]:
outputfolder = '/home/da/tools/synthpop/demos/outputs/'
hdfpath = outputfolder+"starter%s_"%(starter_ver)+time.strftime("%Y%m%d-%H%M")



# Synthesize by county 

In [ ]:
#for whole county
time0=time.time()
print str( datetime.datetime.now())
results=[]

store = pd.HDFStore(hdfpath+'.h5',complevel=9, complib='blosc')

#synthesize_counties(["Livingston County","Oakland County","Washtenaw County",
#"Wayne County","Monroe County","Macomb County","St. Clair County"])

if __name__=='__main__':
    counties=["Wayne County"]

    for county in counties:
        finalcheck=[]
        phases=3
        df_check=None
        
        for phase in range(phases):
            print "\n county = ",county," phase = ", phase, "\n"
            
            pathout = county.replace(" ","_") #output name in hdf
            
            starter = Starter(os.environ["CENSUS"], "MI", county, phase = phase)
           
            households, persons, fit, hh_marginal, pp_marginal = synthesize_all(starter)

            #households.index.name='hh_id' #result[0] is hh
            #households['serialno'] = households['serialno'].map(lambda x: '%f' % x)
            #persons['serialno'] = persons['serialno'].map(lambda x: '%f' % x)
            #results[0].to_csv(outputfolder+county+"_starter%s_"%('3')+"HHs.csv",float_format='%.f') #float_format, 
            #          not use scentific, pplies to all fp numbers 
            #could use results['serialno'] = results['serialno'].map(lambda x: '%f' % x) to change 
            #         output format for serialno only
            
            #HHs and Persons output
            if phase == phases-1:
                store[pathout+'_households'] = households
                store[pathout+'_persons'] = persons

            ##process check tables
            dict_check = output_check({'households':[hh_marginal,households], 
                                       'persons':[pp_marginal,persons]
                                      })


            ##outfiles=[pathout+"_check_households.csv",pathout+"_check_households_bycat.csv", \
            ##          pathout+"_check_persons.csv",pathout+"_check_persons_bycat.csv", \
            ##          pathout+"_check_hhsize.csv"]
            
            outfiles = [pathout+"_check_households", 
                        pathout+"_check_households_bycat", 
                        pathout+"_check_persons", 
                        pathout+"_check_persons_bycat",
                        pathout+"_check_hhsize"]
            
            #set index to check tables and rename columns with phase #
            dfcheck = [dict_check['households'][0],
                       dict_check['households'][1],
                       dict_check['persons'][0],
                       dict_check['persons'][1],
                       dict_check['hhsize']]
            indx = ['state','county','tract','block group','cat_name']
            indxl = [indx, indx+['cat_value']]*2+[indx+['hh_size']]

            for ind, df in zip(indxl, dfcheck):
                df.set_index(ind, inplace=True)
                df.rename(columns = { col:col+"_"+str(phase) for col in df.columns}, inplace = True )

            #join check table from multiple runs            
            if len(finalcheck) == 0:
                finalcheck = dfcheck
            else:
                for i in range(5):
                    finalcheck[i] = pd.merge(finalcheck[i], dfcheck[i], \
                                             left_index = True, right_index = True, how = 'left')

            #create additional id fields for analysis purpose  
            if phase == phases-1:
                for i in range(5):
                    finalcheck[i].reset_index(inplace = True)
                    finalcheck[i]['bg10'] = finalcheck[i]['tract']+finalcheck[i]['block group']
                    finalcheck[i]['geoid10'] = finalcheck[i]['state']+finalcheck[i]['county']+ 
                                                finalcheck[i]['tract']+finalcheck[i]['block group']  
                    store[outfiles[i]]=finalcheck[i]

            if phase == 0:
                dict_check['hhsize7plus'].to_csv("hh7_persons_drawing.csv")
            if phase == 1:
                dict_check['person_factor'].to_csv("hh_size_person_factor.csv")
            if phase == 2:
                df = pd.read_csv("hh7_persons_drawing.csv")
                store[pathout+"_hh7_persons_drawing"] = df
                df = pd.read_csv("hh_size_person_factor.csv")
                store[pathout+"_hh_size_person_factor"] = df               
                #os.rename("hh7_persons_drawing.csv", pathout+"_hh7_persons_drawing.csv")
                #os.rename("hh_size_person_factor.csv", pathout+"_hh_size_person_factor.csv")

                
            newhh, newpop = postprocess(households, persons)
            store[pathout+"sim_households"] = newhh
            store[pathout+"sim_persons"] = newpop
            
            households, persons, marginals, dict_check, newhh, newpop = None, None, None, None, None, None
            

            with open(pathout+'_fit_stats.pkl', 'wb') as f:
                pickle.dump(fit, f)

            #del households,persons,hh_marginal,pp_marginal,dict_check, dfcheck
            
            gc.collect()  
            
store.close()            
print 'total time', datetime.timedelta(seconds=(time.time()-time0))
print str( datetime.datetime.now())

In [ ]:
store.close()

In [ ]:
#combine output if there are multiple hdfs
store0=pd.HDFStore(outputfolder+'starter6_20160823.h5',complevel=9, complib='blosc')
lst_st=[outputfolder+'starter6_20160822-0938.h5',
        outputfolder+'starter6_20160823-0804.h5',
       outputfolder+'starter6_20160823-0922.h5']

for st in  lst_st:
    print st
    sts=pd.HDFStore(st)
    for item in sts:
        print '    ',item
        store0[item[1:]]=sts[item]
    sts.close()
    
store0.close()
print 'done'

# post process Synthesizer results 


In [31]:
hdf_input = r"./outputs/starter6_2016-07-12_region.h5"
hdf_output= hdf_input.replace('.h5', '_syn.h5')
directory = outputfolder

# directory = ""
h_name = 'syn_households'
p_name = 'syn_persons'

syn_hdf = pd.HDFStore(hdf_input, mode="r") 
out_hdf = pd.HDFStore(hdf_output, complevel=9, complib='blosc', mode="w") 

def default_cleaner(df):
    df.columns = [x.lower() for x in df.columns]
    df.index.names = [(None if x is None else x.lower()) for x in df.index.names]
    types = df.apply(lambda x: pd.lib.infer_dtype(x.values))
    for col in types[types == 'unicode'].index:
        df[col] = df[col].astype(str)
    df.columns = [str(c) for c in df.columns]
    return df

def sym_cleaner(df):
    df = default_cleaner(df)
    types = df.dtypes
    for col in types[types == np.int64].index:
        df[col] = df[col].astype(float)

    df["county"] = df.county.astype(int)
    if 'hh_id' in df:
        df.hh_id += df.county * 10**7
    df.index += df.county * 10**7
    return df


if h_name in out_hdf:
    del out_hdf[h_name]

for f in syn_hdf:
    if f.endswith("County_households"):
        gc.collect()
        print f
        data = sym_cleaner(syn_hdf[f][['ADJINC', 'NP', 'block group', 'VEH', 'NOC',
                                       'county', 'age_of_head', 'race_of_head', 'hispanic_head',
                                       'state', 'tract', 'workers',
                                       # for placement
                                       'YBL', 'BLD', 'VALP', 'GRNTP', 'HINCP', "hh_tenure_agehead", "ADJHSG"
                                       ]]
                           )

        data.rename(columns={'hincp': 'income', 
                             'grntp': 'rent',
                             'np': 'persons',
                             'race_of_head': 'race_id',
                             'veh': 'cars',
                             'noc': 'children'
                             }, inplace=True)


        data.loc[data.race_id > 2, 'race_id'] = 4
        data.loc[data.hispanic_head == 'yes', 'race_id'] = 3
        del data['hispanic_head']

        data["income"] *= data["adjinc"] / 1000000.0
        del data['adjinc']

        data["rent"] *= data["adjhsg"] / 1000000.0
        del data['adjhsg']

        data.index.name = "household_id"
        
        #update number of workers from persons table since only family workers counted in HH table
        datap = syn_hdf[f.replace('households', 'persons')][['ESR','AGEP', 'hh_id']]
        datap['worker'] = 0
        datap.loc[datap.ESR.isin(range(1,6)),'worker'] = 1
        data['workers'] = datap.groupby('hh_id').worker.sum()
        
        #update number of childrens(AGEP<18) from persons table since NOC has only
        datap['child'] = 0
        datap.loc[(datap.AGEP<18),'child'] = 1
        data['children'] = datap.groupby('hh_id').child.sum()
        
        data.to_hdf(out_hdf, h_name, format='t', append=True, min_itemsize=2 ** 7)
    
        
if p_name in out_hdf:
    del out_hdf[p_name]

for f in syn_hdf:
    if f.endswith("County_persons"):
        gc.collect()
        print f
        data = sym_cleaner(syn_hdf[f][['county', 'AGEP', 'RELP', 'SEX', 'SPORDER', 'hh_id', 'HISP', 'RAC1P']])
        data.rename(
            columns={'agep': 'age',
                     'rac1p': 'race_id',
                     'relp': 'relate',
                     'sporder': 'member_id',
                     'hh_id': 'household_id'}, inplace=True)
        del data['county']

        data.loc[data.race_id > 2, 'race_id'] = 4
        data.loc[data.hisp > 1, 'race_id'] = 3
        del data['hisp']

        data.index.name = "person_id"
        data.to_hdf(out_hdf, p_name, format='t', append=True, min_itemsize=2 ** 7)


syn_hdf.close()
out_hdf.close()
print 'all done'


/Livingston County_households
/Macomb County_households
/Monroe County_households
/Oakland County_households
/St. Clair County_households
/Washtenaw County_households
/Wayne County_households
/Livingston County_persons
/Macomb County_persons
/Monroe County_persons
/Oakland County_persons
/St. Clair County_persons
/Washtenaw County_persons
/Wayne County_persons
all done


/home/da/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:30: FutureWarning: using '+' to provide set union with Indexes is deprecated, use '|' or .union()


In [30]:
out_hdf.close()

# Synthesize by selected Block Groups

In [ ]:
###for selected BGs
#enable_logging()
time0=time.time()
geoname="problembgs3.csv"
pathout=outputfolder+geoname+"_starter%s_"%(starter_ver)

dfbg=pd.read_csv(geoname,index_col=None,dtype=str) #make sure csv column in text not number, so read as'099' not 99
indd=[ind for i, ind in dfbg.iterrows()]
phases=3
finalcheck=[]
for phase in range(phases):
    print "\nphase = ", phase, "\n"
    starter = Starter(os.environ["CENSUS"], "MI", 'Macomb County', phase=phase)
    households, persons, fit, hh_marginal, pp_marginal = synthesize_all(starter, indexes=indd)
    
    #HHs and Persons output
    if phase == phases-1:
        households.to_csv(pathout+'_households.csv')
        persons.to_csv(pathout+'_persons.csv')
 
    ##process check tables
    dict_check=output_check({'households':[hh_marginal,households], 'persons':[pp_marginal,persons]})
    #print dict_check.keys() #['households', 'persons', 'hhsize', 'person_factor', 'hhsize7plus']
    
    outfiles=[pathout+"_check_households.csv",pathout+"_check_households_bycat.csv", \
              pathout+"_check_persons.csv",pathout+"_check_persons_bycat.csv", \
              pathout+"_check_hhsize.csv"]

    #set index to check tables and rename columns with phase #
    dfcheck=[dict_check['households'][0],dict_check['households'][1], \
             dict_check['persons'][0],dict_check['persons'][1], \
             dict_check['hhsize']]
    indx=['state','county','tract','block group','cat_name']
    indxl=[indx, indx+['cat_value']]*2+[indx+['hh_size']]

    for ind, df in zip(indxl, dfcheck):
        df.set_index(ind, inplace=True)
        df.rename(columns={col:col+"_"+str(phase) for col in df.columns}, inplace=True)

    #join check table from multiple runs            
    if len(finalcheck)==0:
        finalcheck=dfcheck
    else:
        for i in range(5):
            finalcheck[i]=pd.merge(finalcheck[i], dfcheck[i], left_index=True, right_index=True, how='left')
    
    #create additional id fields for analysis purpose  
    if phase == phases-1:
        for i in range(5):
            finalcheck[i].reset_index(inplace=True)
            finalcheck[i]['bg10']=finalcheck[i]['tract']+finalcheck[i]['block group']
            finalcheck[i]['geoid10']=finalcheck[i]['state']+finalcheck[i]['county']+ \
                                        finalcheck[i]['tract']+finalcheck[i]['block group']  
            finalcheck[i].to_csv(outfiles[i])

    if phase == 0:
        dict_check['hhsize7plus'].to_csv("hh7_persons_drawing.csv")
    if phase == 1:
        dict_check['person_factor'].to_csv("hh_size_person_factor.csv")
    if phase == 2:
        os.rename("hh7_persons_drawing.csv", pathout+"_hh7_persons_drawing.csv")
        os.rename("hh_size_person_factor.csv", pathout+"_hh_size_person_factor.csv")
        
    households,persons,marginals,dict_check = None, None, None, None
    
print time.time()-time0